# Week 2 Analysis Notebook

This notebook is for tasks completed in week 2. The initial goal is to expand on the work done on the full spectrum galaxy SED as described in the journal mainly looking at evolution over time (by looking at redshift). 

Need to begin by symlinking data from Simba directory in cuillin to here because now need to access data across redshifts. Sticking to 25 Mpc/h volume with 2x256^3 particles but for all redshifts initially. File 151 is z=0 and they go inversely.

In [ ]:
from pathlib import Path
import re
import numpy as np
import h5py
import fsps
import caesar
import astropy.units as u
from astropy.constants import c
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../src')
from week_2_funcs import get_redshift, list_snapshots, compute_summed_sed_from_appmags, compute_summed_sed_from_absmags

CAT_DIR = Path("/home/sid/Documents/edinburgh/year_4/simba_cosmic_background/data/caesar_catalogues/m25n256")

In [ ]:
### Apparent magnitudes-based SED
snaps = list_snapshots()
snap_nums = [150, 145, 135, 120, 110, 100, 95, 85, 75, 65, 55, 45, 35, 25]  # adjust as desired

plt.figure(figsize=(10, 6))

for snap, path in snaps:
    if snap not in snap_nums:
        continue

    obj = caesar.load(str(path))
    z = get_redshift(obj)

    lam_AA, nuFnu, _ = compute_summed_sed_from_appmags(path)
    plt.loglog(lam_AA, nuFnu, marker="o", linewidth=1.5, markersize=4, label=f"z={z:.2f} (snap {snap})")

# plt.gca().invert_xaxis()  # optional: common in astronomy (short λ on right/left preference)
plt.xlabel(r"$\lambda$ ($\AA$)")
plt.ylabel(r"$\nu F_\nu$ (arb; summed over galaxies)")
plt.title("SED evolution using apparent magnitudes (summed over all galaxies)")
plt.grid(True, alpha=0.3, which="both")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
### Absolute magnitudes-based SED
snaps = list_snapshots()
snap_nums = [150, 145, 135, 120, 110, 100, 95, 85, 75, 65, 55, 45, 35, 25]  # adjust as desired

plt.figure(figsize=(10, 6))

for snap, path in snaps:
    if snap not in snap_nums:
        continue

    obj = caesar.load(str(path))
    z = get_redshift(obj)

    lam_AA, nuFnu, _ = compute_summed_sed_from_absmags(path)
    plt.loglog(lam_AA, nuFnu, marker="o", linewidth=1.5, markersize=4, label=f"z={z:.2f} (snap {snap})")

# plt.gca().invert_xaxis()  # optional: common in astronomy (short λ on right/left preference)
plt.xlabel(r"$\lambda$ ($\AA$)")
plt.ylabel(r"$\nu F_\nu$ (arb; summed over galaxies)")
plt.title("SED evolution using absolute magnitudes (summed over all galaxies)")
plt.grid(True, alpha=0.3, which="both")
plt.legend()
plt.tight_layout()
plt.show()